In [1]:
!pip install nltk
!pip install gensim

In [2]:
import pandas as pd
import numpy as np
import sklearn
import gensim
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import re
from html.parser import HTMLParser

In [3]:
with open('./prepared.pickle', 'rb') as f:
    df = pickle.load(f)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    49159 non-null  int64  
 1   label                 31962 non-null  float64
 2   tweet                 49159 non-null  object 
 3   clean_tweet           49159 non-null  object 
 4   tweet_token           49159 non-null  object 
 5   tweet_token_filtered  49159 non-null  object 
 6   tweet_stemmed         49159 non-null  object 
 7   tweet_lemmatized      49159 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.0+ MB


In [5]:
df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in urð,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, urð]","[model, love, take, time, urð]","[model, love, take, time, urð]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform().  
Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.


- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'.
- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


In [6]:
count_bow_vectorizer = sklearn.feature_extraction.text.CountVectorizer(max_df=0.90, max_features=1000, stop_words='english')

In [7]:
tw_st_all = np.array([' '.join(x) for x in df['tweet_stemmed']])

In [8]:
bow_stemmed = count_bow_vectorizer.fit_transform(tw_st_all)
count_bow_stemmed = pd.DataFrame.sparse.from_spmatrix(columns=count_bow_vectorizer.get_feature_names(), data=bow_stemmed)
count_bow_stemmed.shape

(49159, 1000)

In [9]:
with open('./bow_stemmed.pickle', 'wb') as f:
    pickle.dump(count_bow_stemmed, f)

In [10]:
tw_lem_all = np.array([' '.join(x) for x in df['tweet_lemmatized']])

In [11]:
bow_lemma = count_bow_vectorizer.fit_transform(tw_lem_all)
count_bow_lemma = pd.DataFrame.sparse.from_spmatrix(columns=count_bow_vectorizer.get_feature_names(), data=bow_lemma)

In [12]:
with open('./bow_lemma.pickle', 'wb') as f:
    pickle.dump(bow_lemma, f)

2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.  


- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'.
- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().


In [13]:
tfidf_bow_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.90, max_features=1000, stop_words='english')

In [14]:
bow_stemmed = tfidf_bow_vectorizer.fit_transform(tw_st_all)
tfidf_bow_stemmed = pd.DataFrame.sparse.from_spmatrix(columns=tfidf_bow_vectorizer.get_feature_names(), data=bow_stemmed)
tfidf_bow_stemmed.head()

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,³ð,µð,¹ð,ºï,ºð,¼ð,½ï,½ð,¾ð,ó¾
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
with open('./tfidf_stemmed.pickle', 'wb') as f:
    pickle.dump(tfidf_bow_stemmed, f)

In [16]:
bow_lemma = tfidf_bow_vectorizer.fit_transform(tw_lem_all)
tfidf_bow_lemma = pd.DataFrame.sparse.from_spmatrix(columns=tfidf_bow_vectorizer.get_feature_names(), data=bow_lemma)

In [17]:
with open('./tfidf_lemma.pickle', 'wb') as f:
    pickle.dump(tfidf_bow_lemma, f)

3. Натренируем gensim.models.Word2Vec модель на наших данных.  
Тренировать будем на токенизированных твитах combine_df['tweet_token']  


- Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
- Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.


In [18]:
from gensim.models import Word2Vec

In [19]:
w2v_vectorizer = Word2Vec(corpus_file='./corpus', size=200, window=5, min_count=1, sg=1, hs=0, negative=10, workers=32, seed=34)

In [ ]:
w2v_vectorizer.train(df['tweet_token'], total_examples=df['tweet_token'].shape[0], epochs=20)

In [ ]:
w2v_vectorizer.save('w2v_vectorizer.model')

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [ ]:
w2v_vectorizer.most_similar(positive="dinner")

In [ ]:
w2v_vectorizer.most_similar("trump")

5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
word = 'food'
model = w2v_vectorizer

arr = np.empty((0,200), dtype='f')
word_labels = [word]

close_words = model.similar_by_word(word)
not_close_words = ['dog', 'banana', 'car', 'computer']

arr = np.append(arr, np.array([model[word]]), axis=0)
for wrd_score in close_words:
    wrd_vector = model[wrd_score[0]]
    word_labels.append(wrd_score[0])
    arr = np.append(arr, np.array([wrd_vector]), axis=0)
    
for wrd in not_close_words:
    wrd_vector = model[wrd]
    word_labels.append(wrd)
    arr = np.append(arr, np.array([wrd_vector]), axis=0)

tsne = TSNE(n_components=2, random_state=42)
np.set_printoptions(suppress=True)
Y = tsne.fit_transform(arr)

x_coords = Y[:, 0]
y_coords = Y[:, 1]
# display scatter plot
plt.scatter(x_coords, y_coords)

for label, x, y in zip(word_labels, x_coords, y_coords):
    plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
# plt.xlim(x_coords.min()+0.5, x_coords.max()+0.5)
# plt.ylim(y_coords.min()+0.5, y_coords.max()+0.5)
plt.show()

**.. почему так плохо?**

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).

In [ ]:
data = []

for i in range(df['tweet_token_filtered'].shape[0]):
    vec = []

    for word in df['tweet_token_filtered'][0]:
        try:
            vec.append(model[word])
        except:
            pass
        
    np_vec = np.array(vec)
    data.append(list(np_vec.sum(axis=0)/np_vec.shape[0]))

In [ ]:
np.array(data).shape

In [ ]:
columns=[x for x in range(200)]
wordvec_df = pd.DataFrame(data, columns=columns)

In [ ]:
wordvec_df.head()